<a href="https://colab.research.google.com/github/msbrasil/Algoritmos/blob/main/localize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install python-docx
!pip install docx2pdf
!pip install exceptions
!pip install PyPDF2
!pip install pydrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Automação de fichas de saneamento

pacotes


In [ ]:
import pandas as pd         # pacote para manipular base de dados
from docx import Document   # pacote para manipulação de arquivos docx
import re                   # pacote para utilização de expressões regulares (regex)
import docx2pdf             # pacote para conversão de docx para pdf
import locale               # dependência

In [ ]:
#locale.setlocale(locale.LC_ALL, 'de_DE.utf-8')

In [ ]:
snis = pd.read_excel('/content/drive/MyDrive/automacao-localize/snis.xlsx')

### Selecionar as variáveis de interesse

In [ ]:
sanepar = snis.filter([
    'Município',
    'Prestador',
    'GE017 - Ano de vencimento da delegação de abastecimento de água',
    'POP_TOT - População total do município do ano de referência (Fonte: IBGE):',
    'AG001 - População total atendida com abastecimento de água',
    'Tipo de serviço',
    'AG026 - População urbana atendida com abastecimento de água',
    'AG002 - Quantidade de ligações ativas de água',
    'AG006 - Volume de água produzido',
    'AG011 - Volume de água faturado',
    'AG010 - Volume de água consumido',
    'QD002 - Quantidades de paralisações no sistema de distribuição de água',
    'IN076 - Incidência das análises de turbidez fora do padrão',
    'QD027 - Quantidade de amostras para coliformes totais com resultados fora do padrão',
    'IN079 - Índice de conformidade da quantidade de amostras - cloro residual',
    'ES001 - População total atendida com esgotamento sanitário',
    'ES002 - Quantidade de ligações ativas de esgotos',
    'ES005 - Volume de esgotos coletado',
    'ES007 - Volume de esgotos faturado',
    'ES006 - Volume de esgotos tratado',
    'ES004 - Extensão da rede de esgotos',
])

In [ ]:
sanepar.fillna(0, inplace=True)


In [ ]:
sanepar['Tipo de serviço'] = sanepar['Tipo de serviço'].str.strip()

In [ ]:
sanepar['POP_TOT - População total do município do ano de referência (Fonte: IBGE):'] = sanepar[ 'POP_TOT - População total do município do ano de referência (Fonte: IBGE):'].apply(lambda x: locale.format('%d', x, 1))
sanepar['AG001 - População total atendida com abastecimento de água'] = sanepar['AG001 - População total atendida com abastecimento de água'].apply(lambda x: locale.format('%d', x, 1))
sanepar['AG026 - População urbana atendida com abastecimento de água'] = sanepar['AG026 - População urbana atendida com abastecimento de água'].apply(lambda x: locale.format('%d', x, 1))
sanepar['AG002 - Quantidade de ligações ativas de água'] = sanepar['AG002 - Quantidade de ligações ativas de água'].apply(lambda x: locale.format('%d', x, 1))
sanepar['GE017 - Ano de vencimento da delegação de abastecimento de água'] = sanepar['GE017 - Ano de vencimento da delegação de abastecimento de água'].astype(int)

<ipython-input-27-bb46a2f52880>:1: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  sanepar['POP_TOT - População total do município do ano de referência (Fonte: IBGE):'] = sanepar[ 'POP_TOT - População total do município do ano de referência (Fonte: IBGE):'].apply(lambda x: locale.format('%d', x, 1))
<ipython-input-27-bb46a2f52880>:2: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  sanepar['AG001 - População total atendida com abastecimento de água'] = sanepar['AG001 - População total atendida com abastecimento de água'].apply(lambda x: locale.format('%d', x, 1))
<ipython-input-27-bb46a2f52880>:3: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  sanepar['AG026 - População urbana atendida com abastecimento de água'] = sanepar['AG026 - População urbana atendida com abastecimento d

In [ ]:
sanepar_agua = sanepar.loc[sanepar['Tipo de serviço'] == ('Água')]

In [ ]:
dict_agua = sanepar_agua.to_dict(orient='records')

In [ ]:
sanepar_agua_esgoto = sanepar.loc[sanepar['Tipo de serviço'] == ('Água e Esgoto')]

In [ ]:
dict_agua_esgoto = sanepar_agua_esgoto.to_dict(orient='records')

In [ ]:
import subprocess

try:
    from comtypes import client
except ImportError:
    client = None

def doc2pdf(doc):
    """
    convert a doc/docx document to pdf format
    :param doc: path to document
    """
    doc = os.path.abspath(doc) # bugfix - searching files in windows/system32
    if client is None:
        return doc2pdf_linux(doc)
    name, ext = os.path.splitext(doc)
    try:
        word = client.CreateObject('Word.Application')
        worddoc = word.Documents.Open(doc)
        worddoc.SaveAs(name + '.pdf', FileFormat=17)
    except Exception:
        raise
    finally:
        worddoc.Close()
        word.Quit()

      
def doc2pdf_linux(doc):
    """
    convert a doc/docx document to pdf format (linux only, requires libreoffice)
    :param doc: path to document
    """
    cmd = 'libreoffice --convert-to pdf'.split() + [doc]
    p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    p.wait(timeout=10)
    stdout, stderr = p.communicate()
    if stderr:
        raise subprocess.SubprocessError(stderr)

In [ ]:
for dicio in dict_agua_esgoto:
    cidade = dicio['Município']
    regex1 = re.compile("<Município>")
    replace1 = f"{dicio['Município']}"
    regex2 = re.compile("<ano de vencimento>")
    replace2 = f"{dicio['GE017 - Ano de vencimento da delegação de abastecimento de água']}"
    regex3 = re.compile("<pop total do munícipio>")
    replace3 = f"{dicio['POP_TOT - População total do município do ano de referência (Fonte: IBGE):']}"
    regex4 = re.compile("<pop urbana>")
    replace4 = f"{dicio['AG026 - População urbana atendida com abastecimento de água']}"
    regex5 = re.compile("<ligações ativas>")
    replace5 = f"{dicio['AG002 - Quantidade de ligações ativas de água']}"
    regex6 = re.compile("<produzido>")
    replace6 = f"{dicio['AG006 - Volume de água produzido']}"
    regex7 = re.compile("<faturado>")
    replace7 = f"{dicio['AG011 - Volume de água faturado']}"
    regex8 = re.compile("<consumido>")
    replace8 = f"{dicio['AG010 - Volume de água consumido']}"
    regex9 = re.compile("<paralisações>")
    replace9 = f"{dicio['QD002 - Quantidades de paralisações no sistema de distribuição de água']}"
    regex10 = re.compile("<turbidez>")
    replace10 = f"{dicio['IN076 - Incidência das análises de turbidez fora do padrão']}"
    regex11 = re.compile("<coliformes>")
    replace11 = f"{dicio['QD027 - Quantidade de amostras para coliformes totais com resultados fora do padrão']}"
    regex12 = re.compile("<cloro>")
    replace12 = f"{dicio['IN079 - Índice de conformidade da quantidade de amostras - cloro residual']}"
    regex13 = re.compile("<pop total agua>")
    replace13 = f"{dicio['AG001 - População total atendida com abastecimento de água']}" 
    regex14 = re.compile("<pop total esgoto>")
    replace14 = f"{dicio['ES001 - População total atendida com esgotamento sanitário']}"
    regex15 = re.compile("<lig ativo esgoto>")
    replace15 = f"{dicio['ES002 - Quantidade de ligações ativas de esgotos']}"
    regex16 = re.compile("<coletado esgoto>")
    replace16 = f"{dicio['ES005 - Volume de esgotos coletado']}"
    regex17 = re.compile("<faturado esgoto>")
    replace17 = f"{dicio['ES007 - Volume de esgotos faturado']}"
    regex18 = re.compile("<tratado esgoto>")
    replace18 = f"{dicio['ES006 - Volume de esgotos tratado']}"
    regex19 = re.compile("<rede esgoto>")
    replace19 = f"{dicio['ES004 - Extensão da rede de esgotos']}"



    filename = "/content/drive/MyDrive/automacao-localize/agua e esgoto.docx"
    doc = Document(filename)

    docx_replace_regex(doc, regex1 , replace1)
    docx_replace_regex(doc, regex2 , replace2)
    docx_replace_regex(doc, regex3 , replace3)
    docx_replace_regex(doc, regex4 , replace4)
    docx_replace_regex(doc, regex5 , replace5)
    docx_replace_regex(doc, regex6 , replace6)
    docx_replace_regex(doc, regex7 , replace7)
    docx_replace_regex(doc, regex8 , replace8)
    docx_replace_regex(doc, regex9 , replace9)
    docx_replace_regex(doc, regex10 , replace10)
    docx_replace_regex(doc, regex11 , replace11)
    docx_replace_regex(doc, regex12 , replace12)
    docx_replace_regex(doc, regex13 , replace13)
    docx_replace_regex(doc, regex14 , replace14)
    docx_replace_regex(doc, regex15 , replace15)
    docx_replace_regex(doc, regex16 , replace16)
    docx_replace_regex(doc, regex17 , replace17)
    docx_replace_regex(doc, regex18 , replace18)
    docx_replace_regex(doc, regex19 , replace19)
    


    abiword --to=pdf f'/content/drive/MyDrive/automacao-localize/words/{cidade}.docx'

SyntaxError: ignored

In [ ]:
def convert(input_path, output_path=None, keep_active=False):
    paths = resolve_paths(input_path, output_path)
    if sys.platform == "darwin":
        return macos(paths, keep_active)
    elif sys.platform == "win32":
        return windows(paths, keep_active)
    else:
#        raise NotImplementedError(
#            "docx2pdf is not implemented for linux as it requires Microsoft Word to be installed"
#        )
        return windows(paths, keep_active)

In [ ]:
import subprocess

try:
    from comtypes import client
except ImportError:
    client = None

def doc2pdf(doc):
    """
    convert a doc/docx document to pdf format
    :param doc: path to document
    """
    doc = os.path.abspath(doc) # bugfix - searching files in windows/system32
    try:
        word = client.CreateObject('Word.Application')
        worddoc = word.Documents.Open(doc)
        worddoc.SaveAs(name + '.pdf', FileFormat=17)
    except Exception:
        raise
    finally:
        worddoc.Close()
        word.Quit()